## 성과 데이터 크롤링
- traditional
- advanced
- opponent
- defense


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time


## traditional

In [89]:

# 크롬 드라이버 경로 설정
chrome_driver_path = 'C:/Users/minye/Downloads/chromedriver-win64/chromedriver.exe'

# Service 객체를 사용하여 크롬 드라이버 경로 전달
service = Service(executable_path=chrome_driver_path)

# 웹드라이버 설정
driver = webdriver.Chrome(service=service)

# 크롤링할 시즌 목록 (2000-2001 시즌부터 2022-2023 시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2001, 2002)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    # 시즌별 URL
    url = f"https://www.nba.com/stats/players/traditional?Season={season}&SeasonType=Regular+Season"
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업이 있으면 닫기 (쿠키 동의 팝업)
    try:
        # 쿠키 동의 버튼의 XPath 클릭
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 최대 페이지 설정 (필요에 따라 조정 가능)
    max_pages = 5  # 각 시즌당 5페이지까지만 크롤링 (필요에 따라 변경 가능)

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

        # 페이지 반복 처리 (끝 페이지까지 자동으로 반복)
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 필요한 테이블 부분 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        # 열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:29]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][1:]
            data.append(season)  # 각 데이터에 시즌 정보 추가
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지 버튼 클릭 시도
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            # '다음' 버튼이 비활성화된 경우 (마지막 페이지인 경우) 반복 중단
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break

            # 다음 페이지로 이동
            next_button.click()
            time.sleep(5)  # 페이지가 로드될 시간을 줌
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    all_data = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")

# 작업이 끝나면 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
print(all_data.head())  # 추출된 데이터의 앞부분만 확인
# all_data.to_csv('nba_traditional_stats_2000_2023.csv', index=False)  # CSV 파일로 저장


Cookie popup closed.
Reached the last page or error occurred in season 2001-02: Message: 

Season 2001-02 data collected.
             Player Team Age  GP   W   L   Min   PTS   FGM   FGA  ...  AST  \
0     Allen Iverson  PHI  27  60  36  24  43.7  31.4  11.1  27.8  ...  5.5   
1  Shaquille O'Neal  LAL  30  67  51  16  36.1  27.2  10.6  18.3  ...  3.0   
2       Paul Pierce  BOS  24  82  49  33  40.2  26.1   8.6  19.5  ...  3.2   
3     Tracy McGrady  ORL  23  76  43  33  38.3  25.6   9.4  20.9  ...  5.3   
4        Tim Duncan  SAS  26  82  58  24  40.6  25.5   9.3  18.3  ...  3.7   

   TOV  STL  BLK   PF    FP   DD2  TD3  +/-   Season  
0  4.0  2.8  0.2  1.7  50.1   4.0  1.0  4.7  2001-02  
1  2.6  0.6  2.0  3.0  49.9  40.0  0.0  8.5  2001-02  
2  2.9  1.9  1.0  2.9  45.0  17.0  0.0  3.8  2001-02  
3  2.5  1.6  1.0  1.8  48.0  24.0  1.0  3.9  2001-02  
4  3.2  0.7  2.5  2.6  52.8  67.0  0.0  6.8  2001-02  

[5 rows x 30 columns]


## advanced

In [99]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
chrome_driver_path = 'C:/Users/minye/Downloads/chromedriver-win64/chromedriver.exe'

# Service 객체를 사용하여 크롬 드라이버 경로 전달
service = Service(executable_path=chrome_driver_path)

# 웹드라이버 설정
driver = webdriver.Chrome(service=service)

# 크롤링할 시즌 목록 (2000-2001 시즌부터 2022-2023 시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2001, 2002)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    # 시즌별 URL
    url = f"https://www.nba.com/stats/players/advanced?Season={season}&SeasonType=Regular+Season"
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업이 있으면 닫기 (쿠키 동의 팝업)
    try:
        # 쿠키 동의 버튼의 XPath 클릭
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

    # 페이지 반복 처리 (끝 페이지까지 자동으로 반복)
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 필요한 테이블 부분 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        # 열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:23]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][1:]
            data.append(season)  # 각 데이터에 시즌 정보 추가
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지 버튼 클릭 시도
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            # '다음' 버튼이 비활성화된 경우 (마지막 페이지인 경우) 반복 중단
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break

            # 다음 페이지로 이동
            next_button.click()
            time.sleep(5)  # 페이지가 로드될 시간을 줌
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    all_data = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")

# 작업이 끝나면 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
print(all_data.head())  # 추출된 데이터의 앞부분만 확인
# all_data.to_csv('nba_advanced_stats_2000_2023.csv', index=False)  # CSV 파일로 저장


Cookie popup closed.
Reached the last page or error occurred in season 2001-02: Message: 

Season 2001-02 data collected.
             PLAYER TEAM AGE  GP   W   L   MIN OFFRTG DEFRTG NETRTG  ...  \
0         Bo Outlaw  PHX  31  83  37  46  23.2  100.6   98.4    2.2  ...   
1        Jalen Rose  CHI  29  83  35  48  38.0  102.6  104.6   -2.0  ...   
2    Aaron Williams  NJN  30  82  52  30  18.9   99.7   95.1    4.7  ...   
3    Alvin Williams  TOR  27  82  42  40  35.8  101.4  101.8   -0.5  ...   
4  Andrei Kirilenko  UTA  21  82  44  38  26.3  105.4   97.3    8.1  ...   

  DREB%  REB% TO RATIO  EFG%   TS%  USG%   PACE   PIE  POSS   Season  
0  11.1   9.5     17.1  55.7  54.7  10.1  92.19   7.1  3691  2001-02  
1  10.4   6.0      9.3  48.5  52.8  25.7  93.94  11.3  6174  2001-02  
2  14.2  11.1     11.6  52.6  56.8  17.7  92.88  10.2  2990  2001-02  
3   7.7   4.8      9.1  44.7  47.0  18.1  90.91   9.8  5553  2001-02  
4  12.7  10.0     10.7  47.0  55.3  18.8  93.24  12.5  4187  2001-

## usage

In [113]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
chrome_driver_path = 'C:/Users/minye/Downloads/chromedriver-win64/chromedriver.exe'

# Service 객체를 사용하여 크롬 드라이버 경로 전달
service = Service(executable_path=chrome_driver_path)

# 웹드라이버 설정
driver = webdriver.Chrome(service=service)

# 크롤링할 시즌 목록 (2000-2001 시즌부터 2022-2023 시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2001, 2002)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    # 시즌별 URL
    url = f"https://www.nba.com/stats/players/usage?Season={season}&SeasonType=Regular+Season"
    # url=f"https://www.nba.com/stats/players/usage?SeasonType=Regular+Season&Season=2023-24"
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업이 있으면 닫기 (쿠키 동의 팝업)
    try:
        # 쿠키 동의 버튼의 XPath 클릭
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

    # 페이지 반복 처리 (끝 페이지까지 자동으로 반복)
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 필요한 테이블 부분 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        # 열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:25]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][1:]
            data.append(season)  # 각 데이터에 시즌 정보 추가
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지 버튼 클릭 시도
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            # '다음' 버튼이 비활성화된 경우 (마지막 페이지인 경우) 반복 중단
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break

            # 다음 페이지로 이동
            next_button.click()
            time.sleep(5)  # 페이지가 로드될 시간을 줌
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    all_data = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")

# 작업이 끝나면 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
print(all_data.head())  # 추출된 데이터의 앞부분만 확인
# all_data.to_csv('nba_usage_stats_2000_2023.csv', index=False)  # CSV 파일로 저장


Cookie popup closed.
Reached the last page or error occurred in season 2001-02: Message: 

Season 2001-02 data collected.
             Player TEAM AGE  GP   W   L     MIN  USG%  %FGM  %FGA  ...  %REB  \
0     Allen Iverson  PHI  27  60  36  24  2624.0  36.5  34.8  38.2  ...  10.9   
1   Vinny Del Negro  PHX  35   2   1   1     5.0  36.4  25.0  44.4  ...   0.0   
2    Michael Jordan  WAS  39  60  30  30  2089.0  35.6  34.4  36.8  ...  18.7   
3  Shaquille O'Neal  LAL  30  67  51  16  2421.0  31.4  35.9  29.8  ...  31.4   
4  Jerry Stackhouse  DET  27  76  45  31  2686.0  31.4  27.2  30.4  ...  14.6   

   %AST  %TOV  %STL  %BLK %BLKA   %PF %PFD  %PTS   Season  
0  30.2  29.2  33.7   5.4  29.4   9.8  0.1  36.7  2001-02  
1   100   0.0   0.0   0.0   0.0   0.0  0.0  16.7  2001-02  
2  33.1  32.7  30.1  14.4  27.5  13.3  0.1  33.2  2001-02  
3  16.4  26.7  10.8  47.7  28.4  18.4  0.0  34.9  2001-02  
4  33.9  33.3  17.0   9.3  31.8  14.3  0.0  30.8  2001-02  

[5 rows x 26 columns]


## defense

In [119]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
chrome_driver_path = 'C:/Users/minye/Downloads/chromedriver-win64/chromedriver.exe'

# Service 객체를 사용하여 크롬 드라이버 경로 전달
service = Service(executable_path=chrome_driver_path)

# 웹드라이버 설정
driver = webdriver.Chrome(service=service)

# 크롤링할 시즌 목록 (2000-2001 시즌부터 2022-2023 시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2001, 2002)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    # 시즌별 URL
    url = f"https://www.nba.com/stats/players/defense?Season={season}&SeasonType=Regular+Season"
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업이 있으면 닫기 (쿠키 동의 팝업)
    try:
        # 쿠키 동의 버튼의 XPath 클릭
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

    # 페이지 반복 처리 (끝 페이지까지 자동으로 반복)
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 필요한 테이블 부분 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        # 열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:20]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][1:]
            data.append(season)  # 각 데이터에 시즌 정보 추가
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지 버튼 클릭 시도
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            # '다음' 버튼이 비활성화된 경우 (마지막 페이지인 경우) 반복 중단
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break

            # 다음 페이지로 이동
            next_button.click()
            time.sleep(5)  # 페이지가 로드될 시간을 줌
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    all_data = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")

# 작업이 끝나면 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
print(all_data.head())  # 추출된 데이터의 앞부분만 확인
# all_data.to_csv('nba_defense_stats_2000_2023.csv', index=False)  # CSV 파일로 저장


Cookie popup closed.
Reached the last page or error occurred in season 2001-02: Message: 

Season 2001-02 data collected.
            Player TEAM AGE  GP   W   L   MIN DEF RTG DREB DREB%  ...  STL  \
0    Allen Iverson  PHI  27  60  36  24  43.7    96.6  3.8   8.4  ...  2.8   
1       Tim Duncan  SAS  26  82  58  24  40.6    96.7  9.4  22.7  ...  0.7   
2  Dikembe Mutombo  PHI  36  80  43  37  36.3    95.5  7.6  20.7  ...  0.4   
3      Eddie Jones  MIA  30  81  36  45  39.0    97.1  3.9  10.2  ...  1.4   
4        Eric Snow  PHI  29  61  35  26  36.5    96.8  3.0   8.1  ...  1.6   

   STL%  BLK  %BLK OPP PTSOFF TOV OPP PTS2ND CHANCE OPP PTSFB OPP PTSPAINT  \
0  33.7  0.2   5.4           13.1              11.3      12.2         31.2   
1  12.3  2.5  44.2           11.9              11.4      10.0         30.7   
2   6.0  2.4  68.1           11.0               8.3       9.9         22.7   
3  26.5  1.0  20.5           12.6               9.7       8.7         26.0   
4  23.8  0.1   4.6 

In [123]:
merged_data.to_csv('nba_defense_stats_final.csv', index=False)  # CSV 파일로 저장

## 2023-2024시즌 데이터 크롤링

## team-box-score

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
chrome_driver_path = 'C:/Users/minye/Downloads/chromedriver-win64/chromedriver.exe'

# Service 객체를 사용하여 크롬 드라이버 경로 전달
service = Service(executable_path=chrome_driver_path)

# 웹드라이버 설정
driver = webdriver.Chrome(service=service)

# 크롤링할 시즌 목록 (2000-2001 시즌부터 2022-2023 시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2023, 2024)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    # 시즌별 URL
    url = f"https://www.nba.com/stats/teams/boxscores?Season={season}&SeasonType=Regular+Season"
    url
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업이 있으면 닫기 (쿠키 동의 팝업)
    try:
        # 쿠키 동의 버튼의 XPath 클릭
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 최대 페이지 설정 (필요에 따라 조정 가능)
    max_pages = 5  # 각 시즌당 5페이지까지만 크롤링 (필요에 따라 변경 가능)

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

        # 페이지 반복 처리 (끝 페이지까지 자동으로 반복)
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 필요한 테이블 부분 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        # 열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:25]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][0:]
            data.append(season)  # 각 데이터에 시즌 정보 추가
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지 버튼 클릭 시도
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            # '다음' 버튼이 비활성화된 경우 (마지막 페이지인 경우) 반복 중단
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break

            # 다음 페이지로 이동
            next_button.click()
            time.sleep(5)  # 페이지가 로드될 시간을 줌
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    all_data = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")

# 작업이 끝나면 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
print(all_data.head())  # 추출된 데이터의 앞부분만 확인
# all_data.to_csv('nba_traditional_stats_2000_2023.csv', index=False)  # CSV 파일로 저장


Cookie popup closed.
Reached the last page or error occurred in season 2023-24: Message: 

Season 2023-24 data collected.
  Team     Match Up   Game Date W/L  MIN  PTS FGM  FGA   FG% 3PM  ... OREB  \
0  HOU    HOU @ LAC  04/14/2024   W  240  116  50  103  48.5  11  ...   17   
1  ATL    ATL @ IND  04/14/2024   L  240  115  39   89  43.8  12  ...    9   
2  TOR    TOR @ MIA  04/14/2024   L  240  103  38   90  42.2   9  ...   13   
3  MIA  MIA vs. TOR  04/14/2024   W  240  118  46   87  52.9   9  ...   13   
4  MIL    MIL @ ORL  04/14/2024   L  240   88  30   75  40.0   7  ...    7   

  DREB REB AST STL BLK TOV  PF  +/-   Season  
0   42  59  31   7   8  18  13   11  2023-24  
1   23  32  25   6   5  17  12  -42  2023-24  
2   28  41  23   6   3  19  21  -15  2023-24  
3   33  46  29  13   7  19  16   15  2023-24  
4   27  34  16  10   4  17  18  -25  2023-24  

[5 rows x 25 columns]


In [40]:
a.head()
a.shape

(1230, 19)

In [44]:
# all_data.to_csv('team_difference.csv',index=False)
# pd.read_csv('team_difference.csv')

In [52]:
# b.head()
b.shape

(12723, 27)

## player-boxscore

In [4]:
# player box score
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
chrome_driver_path = 'C:/Users/minye/Downloads/chromedriver-win64/chromedriver.exe'

# Service 객체를 사용하여 크롬 드라이버 경로 전달
service = Service(executable_path=chrome_driver_path)

# 웹드라이버 설정
driver = webdriver.Chrome(service=service)

# 크롤링할 시즌 목록 (2000-2001 시즌부터 2022-2023 시즌까지)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(2023, 2024)]

# 전체 데이터를 저장할 데이터프레임
all_data = pd.DataFrame()

# 각 시즌에 대해 크롤링
for season in seasons:
    # 시즌별 URL
    url = f"https://www.nba.com/stats/players/boxscores?Season={season}&SeasonType=Regular+Season"
    driver.get(url)
    
    # 페이지 로드 대기
    time.sleep(5)

    # 팝업이 있으면 닫기 (쿠키 동의 팝업)
    try:
        # 쿠키 동의 버튼의 XPath 클릭
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        cookie_button.click()
        print("Cookie popup closed.")
    except Exception as e:
        print(f"No cookie popup to close: {e}")

    # 열 이름 설정
    columns = []

    # 최대 페이지 설정 (필요에 따라 조정 가능)
    max_pages = 5  # 각 시즌당 5페이지까지만 크롤링 (필요에 따라 변경 가능)

    # 시즌 데이터 저장용 데이터프레임
    season_data = pd.DataFrame(columns=columns)

        # 페이지 반복 처리 (끝 페이지까지 자동으로 반복)
    while True:
        # 페이지의 전체 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 필요한 테이블 부분 찾기
        table = soup.find('section', {'class': 'Block_block__62M07 nba-stats-content-block'})

        # 열 이름 추출 (첫 페이지에서만 수행)
        if not columns:
            header_row = table.find('tr', {'class': 'Crom_headers__mzI_m'})
            columns = [th.get_text(strip=True) for th in header_row.find_all('th') if th.get_text(strip=True)][:27]
            columns.append('Season')  # 시즌 정보를 추가
            season_data = pd.DataFrame(columns=columns)

        # 데이터 행 추출
        rows = table.find('tbody', {'class': 'Crom_body__UYOcU'}).find_all('tr')

        # 각 행의 데이터 추출
        for row in rows:
            data = [td.get_text(strip=True) for td in row.find_all('td')][0:]
            data.append(season)  # 각 데이터에 시즌 정보 추가
            season_data = pd.concat([season_data, pd.DataFrame([data], columns=columns)], ignore_index=True)

        # 다음 페이지 버튼 클릭 시도
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]'))
            )
            # '다음' 버튼이 비활성화된 경우 (마지막 페이지인 경우) 반복 중단
            if "disabled" in next_button.get_attribute("class"):
                print(f"Reached the last page for season {season}.")
                break

            # 다음 페이지로 이동
            next_button.click()
            time.sleep(5)  # 페이지가 로드될 시간을 줌
        except Exception as e:
            print(f"Reached the last page or error occurred in season {season}: {e}")
            break

    # 해당 시즌의 데이터를 전체 데이터프레임에 추가
    all_data = pd.concat([all_data, season_data], ignore_index=True)
    print(f"Season {season} data collected.")

# 작업이 끝나면 드라이버 종료
driver.quit()

# 결과 출력 또는 저장
# print(all_data.head())  # 추출된 데이터의 앞부분만 확인
all_data.to_csv('plyaer_box_data.csv', index=False)  # CSV 파일로 저장


Cookie popup closed.
Reached the last page or error occurred in season 2023-24: Message: 

Season 2023-24 data collected.
